In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
countryH3 = pd.DataFrame()
countryBlock = pd.DataFrame()

folderList = [x[0] for x in os.walk('data')][1:]

for folder in folderList:
    fileList = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(folder)) for f in fn]
    fileList = [item for item in fileList if item.endswith(".csv")]
    stateData = pd.DataFrame()
    for file in tqdm(fileList):
        df = pd.DataFrame()
        df = pd.read_csv(f'{file}')
        os.remove(file)
        stateName = df.state_usps.unique()[0].lower()
        columnsToDrop = ['frn',
            'provider_id',
            'brand_name',
            'technology',
            'max_advertised_download_speed',
            'max_advertised_upload_speed',
            'low_latency',
            'business_residential_code',
            'state_usps'
        ]
        df = df.drop(columns=columnsToDrop)
        stateData = pd.concat([stateData, df], ignore_index=True)
    print(stateName)
    stateData.drop_duplicates(inplace=True)
    stateData.reset_index(drop=True, inplace=True)
    dfH3 = pd.DataFrame(stateData)
    dfBlock = pd.DataFrame(stateData)
    dfH3.drop(columns=['block_geoid'], inplace=True)
    dfH3.drop_duplicates(inplace=True)
    dfBlock.drop(columns=['h3_res8_id'], inplace=True)
    dfBlock.drop_duplicates(inplace=True)
    dfH3.drop(columns=['location_id'], inplace=True)
    dfBlock.drop(columns=['location_id'], inplace=True)
    dfH3 = dfH3.groupby('h3_res8_id').size().reset_index().rename(columns={0: 'bsls'})
    dfBlock = dfBlock.groupby('block_geoid').size().reset_index().rename(columns={0: 'bsls'})
    countryH3 = pd.concat([countryH3, dfH3], ignore_index=True)
    countryBlock = pd.concat([countryBlock, dfBlock], ignore_index=True)
    print(countryH3[countryH3['h3_res8_id'] == '882ab269d7fffff'])
    dfH3.to_csv(f'{stateName}H3Bsls.csv', index=False)
    dfBlock.to_csv(f'{stateName}BlockBsls.csv', index=False)

# countryBlock = countryBlock.groupby('block_geoid').sum().reset_index()
# countryH3 = countryH3.groupby('h3_res8_id').sum().reset_index()

  0%|          | 0/8 [00:00<?, ?it/s]


KeyboardInterrupt: 